# **\[Note\]** 우분투 포맷 및 개발용 서버 셋팅 (old)

신록예찬  
2000-01-01

# About this doc

`-` 우분투에서 여러가지 개발환경을 설정하는 방법을 포스팅 하겠다.

`-` 이 포스트는 우분투를 메인OS(사무용+연구용)로 사용하고 싶은 사람,
우분투를 활용하여 개발용 서버를 구축하고 싶은 사람에게 모두 유용한다.

`-` 이 포스트는 2080 이상의 GPU를 활용한 학습을 원하는 사람에게
유용하다.

`-` 이 포스트는 R과 파이썬을 동시에 쓰는 사람에게 유용하다.

`-` 이 포스트는 Rstudio, Jupyter Lab을 동시에 쓰는 사람에게 유용하다.

`-` 매년 조금씩 셋팅방법이 다른것 같다.

`-` 가장 최근에는 2023년 3월8일에 이 블로그 내용으로 셋팅해보았음.

# 우분투설치

`-` 22.04부터는 파티션 나누지 않고 그냥 설치해도 잘 되는것 같다.

# 네트워크 설정

`-` `?`표시있는 아이콘 $\to$ Wired Connected $\to$ Wired Settings $\to$
Connection의 설정 $\to$ IPv4 $\to$ Manual $\to$ Address, Netmask,
Gateway, DNS 설정 $\to$ 네트워크 토글

# 한글설정 (개발용 서버일 경우 생략 가능)

`-` 아래와 같이 커맨드에 친다.

``` default
ibus-setup
```

이걸 치면 `IBus Preferences` 라는 창이 나오는데 여기에서 (1)
`Input Method` 탭 클릭 (2) `Add` 버튼 클릭 (3) `Korean` 선택 (4)
`Hangul` 선택을 한다.

`-` 위의 단계에서 `Korean`이 안보이면 `Language Support`로 가서
한국어팩을 설치하고 리부팅 하면 된다. (보통 실행하자마자 알아서
설치되더라.. 설치가 안되면 `Install / Remove Languages...` 이라는 탭을
클릭해서 설치하자) 리부팅을 꼭 해야한다는 것에 주의하자.

`-` 이제 `Region & Language`로 가서 설정하면 된다.

# 그래픽카드 드라이버설치

`-` 전체적인 내용은
[여기](https://gist.github.com/bendangnuksung/981408031699e0ddc50a6f6fdcf185c2)를
참고하자.

`-` 준비작업

``` default
sudo apt update 
sudo apt install gcc
sudo apt install build-essential
```

`-` 우선 `gedit`를 열고 아래를 복사해서 붙여넣는다.

``` default
blacklist nouveau
options nouveau modeset=0
```

파일이름을 `blacklist-nouveau.conf`로 `home`에 저장

`-` 루트권한획득

``` default
sudo -i
```

아이디와 비밀번호를 입력하고 루트권한을 얻는다.

`-` 아래를 입력한다.

``` default
sudo cp /home/cgb2/blacklist-nouveau.conf /etc/modprobe.d
sudo update-initramfs -u
sudo reboot 
```

`-` 그래픽카드 다운로드: 드라이버 설치파일을 다운받는다.
[앤비디아공식홈페이지](https://www.geforce.com/drivers)에서 다운받자.
OS를 리눅스 64-bit으로 선택하고 검색을 누르면 다운받아진다.

`-` 그래픽키다 설치: 다운받은뒤에는 파일이 있는 폴더로 이동하여

``` default
chmod +x NVIDIA-Linux-x86_64-410.78.run
```

를 실행하자. 보통 `NVI`까지치고 적당히 `탭`을 누르면 알아서 뒷부분이
완성된다. 이 과정은 추후에 드라이버를 실행할수 있도록 권한을 풀어두는
것이다. 그리고 아래를 실행한다.

``` default
sudo ./NVIDIA-Linux-x86_64-410.78.run
```

그 다음 드라이버가 잘 설치되었는지 확인한다.

``` default
nvidia-smi
```

# 아나콘다

`-` (아나콘다 설치) 아나콘다를 다운받은 폴더로 가서 아래와 같이
실행한다.

``` default
bash Anaconda3-2019.03-Linux-x86_64.sh
```

대충 `bash Ana` 정도까지만 치고 `tab`을 누르면 알아서 완성된다.

`-` (환경만들기) 커맨드를 키고 아래를 실행한다.

``` default
(base) conda create -n py38r40 python=3.8
(base) conda create --name py38r40 python=3.8
```

둘 중 아무거나 실행해도 된다. 파이썬 환경이 너무 높으면 나중에
`conda tensorflow-gpu`가 먹히지 않으니 환경을 만들때 파이썬버전을
3.8.x로 하자. (현시점 2021년 2월25일기준 3.9.x이면
`conda tensorflow-gpu` 가 동작하지 않음.)

# ssh연결

`-` 처음에 ssh를 연결하기위해서는 연결**당하는** 컴퓨터에 가서 아래를
실행해야 한다.

    sudo apt install openssh-server

***22번포트 우회하기***

`-` step1: `/etc/ssh/sshd_config` 파일을 연다.

``` default
sudo vi /etc/ssh/sshd_config 
```

`-` step2: Port 22 라고 된 부분의 주석을 풀고 원하는 포트번호 설정

``` default
...

#Port 22
#AddressFamily any
#ListenAddress 0.0.0.0
#ListenAddress ::

...
```

`-` step3: 수정내용을 적용

``` default
sudo systemctl restart ssh.service
```

`-` step4: 수정한 포트로 ssh접속

# 주피터 원격제어

`-` 1단계: 주피랩설치

``` default
(py38) conda install -c conda-forge jupyterlab
```

-   Note: 사실 위에서 주피터랩을 따로 설치안해도 주피터랩이 잘만
    실행된다. 하지만 이렇게하니까 나중에 R커널을 만들기위해
    `IRkernel::installspec()`을 실행할때 에러가 난다.

`-` 2단계: 패스워드 설정

``` default
(py38) jupyter lab --generate-config
(py38) jupyter lab password
```

`-` 3단계: jupyter lab 환경설정

``` default
nano /home/cgb/.jupyter/jupyter_lab_config.py 
```

아래를 변경

``` default
c.ServerApp.ip = '192.168.0.4'
c.ServerApp.port = 1306
c.ServerApp.open_browser = False
```

여기에서 `192.168.0.4` 는 내부아이피다. 고정아피이가 있다면 고정아이피
주소를 쓰면 된다.

# 주피터노트북 원격제어

`-` 1단계: 주피터노트북 설치 (보통 lab을 설치하면 이미 설치되어있음)

``` default
(py38) conda install -c conda-forge notebook 
```

`-` 2단계: 패스워드 설정

``` python
from notebook.auth import passwd
passwd()
Enter password: 
Verify password: 
```

생성된값 (argon 어쩌고..)을 복사

`-` 3단계: 환경설정

``` default
jupyter notebook --generate-config
nano /home/cgb/.jupyter/jupyter_notebook_config.py
```

아이피주소와 패스워드를 바꾼다. (port는 선택, browser도 선택 )

``` default
c.NotebookApp.open_browser = False
c.NotebookApp.ip = '192.168.0.4'
c.NotebookApp.port = 1307
c.NotebookApp.password = ''
```

여기에서 `192.168.0.4` 는 내부아이피다. 고정아피이가 있다면 고정아이피
주소를 쓰면 된다.

> Tip: 주피터노트북과 랩을 양쪽으로 셋팅후 주피터 노트북으로 실행하면
> 2개를 모두 쓸 수 있음

# R설치ver1: (base)에 설치

`-` 설치전: 기존의 R 삭제

``` default
conda remove r-base -y 
sudo apt-get remove r-base-core 
sudo apt purge r-base* r-recommended r-cran-*
sudo apt autoremove
```

`-` [R설치전
준비작업](https://cran.r-project.org/bin/linux/ubuntu/fullREADME.html):
나노에디터를 키고 /etc/apt/sources.list를 연다.

``` default
sudo nano /etc/apt/sources.list
```

화살표로 이동하여 맨아래로 간뒤에 아래중 하나를 추가한다. (나는
focal-cran40으로 추가함)

``` default
deb https://cloud.r-project.org/bin/linux/ubuntu impish-cran40/
deb https://cloud.r-project.org/bin/linux/ubuntu hirsute-cran40/
deb https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/
deb https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/
deb https://cloud.r-project.org/bin/linux/ubuntu xenial-cran40/
```

저장후 나노에디터 종료. 그리고 아래를 실행.

``` default
sudo apt-get update
```

경우에 따라서 아래와 같은 에러메시지가 뜰 수 있다.

``` default
...
W: GPG error: https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease: The following signatures couldn't be verified because the public key is not available: NO_PUBKEY 51716619E084DAB9 
...
```

공개키가 없어서 생기는 에러이므로 아래와 같이 가져온다.

``` default
wget -qO- https://cloud.r-project.org/bin/linux/ubuntu/marutter_pubkey.asc | sudo tee -a /etc/apt/trusted.gpg.d/cran_ubuntu_key.asc
sudo apt-key adv --keyserver keyserver.ubuntu.com --recv-keys E298A3A825C0D65DFD57CBB651716619E084DAB9
#sudo apt-key adv --keyserver hkp://keyserver.ubuntu.com:80 --recv-keys 51716619E084DAB9
```

그리고 다시 아래를 실행

``` default
sudo apt-get update
```

에러가 없이 뭔가 마무리 되어야한다.

``` default
(base) cgb3@cgb3:~$ sudo apt-get update
Ign:1 http://linux.dropbox.com/ubuntu disco InRelease
Hit:2 http://security.ubuntu.com/ubuntu focal-security InRelease  
Hit:3 http://kr.archive.ubuntu.com/ubuntu focal InRelease                                 
Hit:4 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease                
Hit:5 http://linux.dropbox.com/ubuntu disco Release                 
Get:6 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:8 http://kr.archive.ubuntu.com/ubuntu focal-updates InRelease
Get:9 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ Packages [46.4 kB]
Hit:10 http://kr.archive.ubuntu.com/ubuntu focal-backports InRelease
Fetched 50.0 kB in 1s (36.5 kB/s)                   
Reading package lists... Done
```

`-` R설치

``` default
wget http://security.ubuntu.com/ubuntu/pool/main/i/icu/libicu66_66.1-2ubuntu2_amd64.deb
sudo dpkg -i libicu66_66.1-2ubuntu2_amd64.deb
sudo apt-get install r-base
```

`-` tidyverse 설치 (R studio 설치전에 tidyverse 설치해야함)

`-` Rstudio 설치:
<https://www.rstudio.com/products/rstudio/download-server/debian-ubuntu/>

> 우분투22로 설정할것!!

``` default
sudo apt remove rstudio-server
sudo apt-get install gdebi-core
wget https://download2.rstudio.org/server/jammy/amd64/rstudio-server-2022.12.0-353-amd64.deb
sudo gdebi rstudio-server-2022.12.0-353-amd64.deb
```

`-` Rstudio를 설치하면 `~/R/x86_64-conda-linux-gnu-library/4.1`이 새로
생성된다.

-   Rstudio에서 설치한 패키지는 이 폴더에 저장된다.

`-` 주피터와 R커널 연결

``` default
R # sudo R 
install.packages("IRkernel")
IRkernel::installspec()
```

# R설치ver2: (py38r40)에 설치

`-` R설치

``` default
(py38r40) conda install -c conda-forge r-essentials=4.0
```

이러면 콘다환경에는 R이 깔리고 base에는 R이 깔리지 않는다.

`-` 커널연결

콘다환경에서 R을 실행한다. Rstudio가 아니라 커맨드에서 R을 실행해야한다.
그리고 아래를 실행하면 주피터랩과 R환경이 연결된다.

``` r
IRkernel::installspec()
```

이제 주피터랩에서 R kernel을 사용할 수 있다.

# 가상환경에서 Rstudio server 설치 (어려움)

`-` 이제 Rstudio server를 설치하는 방법을 다룬다.

`-` 먼저 Rstudio를 설치한다. 참고로 Rstudio server 설치하는법은
[여기](https://rstudio.com/products/rstudio/download-server/debian-ubuntu/)를
참고하라. 요약하면 터미널에서 아래3줄을 입력하기만 하면된다.

``` default
(py38r40) sudo apt-get install gdebi-core
(py38r40) wget https://download2.rstudio.org/server/bionic/amd64/rstudio-server-1.2.5033-amd64.deb
(py38r40) sudo gdebi rstudio-server-1.2.5033-amd64.deb
```

> Warning: Rstudio 1.3x 이상을 설치하지말고 1.2x를 설치해야 한다.
> 이상하게 1.3x이상은 후에 서술할 Gregor Strurm가 그의 깃허브에서
> 제안하는 방식이 잘 동작하지 않았다. 이는 알려진 문제였고 이를 해결하는
> 해결책을 서술한 스레드가 있어보이긴 했지만 나는 그냥 Rstudio 1.2x를
> 설치하고 쓰는 것을 선택했다.

> Note: 이미 rstudio server 가 다른버전으로 깔려있다면
> `sudo apt remove rstudio-server` 를 통하여 삭제하고 설치하자.

`-` 이제 Rstudio 설치가 끝났다. 설치된 Rstudio를 아나콘다 가상환경에
설치된 R과 연결해보자. 우선 아래를 실행한다.

``` default
(py38r40) sudo apt install uuid
(py38r40) sudo apt install git
(py38r40) git clone https://github.com/grst/rstudio-server-conda.git
```

위에 두줄은 [Gregor
Sturm](https://github.com/grst/rstudio-server-conda)이 만든 어떤
프로그램을 쓰기 위한 사전준비작업이다. 마지막줄을 실행하면 [Gregor
Sturm](https://github.com/grst/rstudio-server-conda)이 만든 프로그램이
다운받아진다. 이게 프로그램 설치가 완료된것이다. 이제 컴퓨터 껐다
킬때마다 아래를 실행한다.

``` default
(py38r40) ./rstudio-server-conda/local/start_rstudio_server.sh 8787 # use any free port number here. 
```

이제 `192.168.0.4:8787` 따위의 주소로 접속하면 `Rstudio`를 쓸 수 있다.
참고로 system-wide Rstudio server를 죽여야 할 때가 있다. 그럴땐 아래
명령을 치면 된다.

``` default
(py38r40) sudo systemctl disable rstudio-server.service
(py38r40) sudo systemctl stop rstudio-server.service
```

# 자주 설치하는 패키지 리스트

`-` 아래를 미리 깔아두자..

``` default
# conda install -c conda-forge notebook
conda install pytorch torchvision torchaudio cudatoolkit=11.3 -c pytorch
conda install -c conda-forge plotly 
pip install plotnine
pip install seaborn
pip install opencv-python
pip install folium
pip install pandas_datareader
conda install -c conda-forge r-essentials=4 
pip install rpy2
conda install -c conda-forge python-graphviz
```

`-` tensorflow-gpu 는 현재(2022-03-06) python=3.10 에서 동작함

``` default
conda create -n py310 python=3.10 
conda activate py310 
conda install -c conda-forge tensorflow-gpu 
```

`-` 아래를 설치하면 좋음

``` default
sudo apt install mc 
```

# 터미널 예쁘게 만들기

`-` zsh 설치 + oh my zsh 설치

``` default
sudo install zsh 
sh -c "$(curl -fsSL https://raw.githubusercontent.com/robbyrussell/oh-my-zsh/master/tools/install.sh)"
```

`-` 테마변경

1.  .zshrc 파일 열기

``` default
nano ~/.zshrc 
```

1.  아래의 내용 수정

``` default
...
ZSH_THEME="agnoster"
...
```

`-` 색상변경

1.  아래의 파일 열기

``` default
cd ~/.oh-my-zsh/themes/
nano agnoster.zsh-theme  
```

1.  내용수정

``` default
...
prompt_dir() {
  prompt_segment 39d $CURRENT_FG '%~'
}
...
```

# sublime text and TeX (개발용 서버일 경우 생략 가능)

`-` ‘Ubuntu Software’에 가서 ’sublime Text’를 치면 다운받을 수 있다.
다운받은뒤에 ’file’ -\> ’open folder’를 활용하여 깃허브의 로칼저장소를
열어두면 편리하다.

`-` 아래를 실행하여 TeX을 깐다.

``` default
sudo apt install texlive-full
```

`-` 이제 sublime과 latex을 연결하여보자.
[여기](https://latextools.readthedocs.io/en/latest/install/#requirements-and-setup)를
참고하자. (1) sublime을 키고 ‘컨트롤+쉬프트+p’를 눌러 ’Install Package
Control’ 선택 (2) 다시 ‘컨트롤+쉬프트+p’ 를 눌러 ‘Package Control:
Install Package’를 실행 (3) 그러면 바로 검색창이 나오는데 거기서
’LaTeXTools’를 입력해서 실행 (4) 다시 ’컨트롤+쉬프트+p’를 누르고
’LaTeXTools: Check system’ 선택. 모두 ’available’이 나오면 잘 설치된
것이다.

`-` `*.tex`파일을 열고 ’컨트롤+b’를 누르자. 처음이면 어떤 메뉴들이
보일텐데 그냥 ’Latex’을 선택하자. 그러면 코딩결과가 pdf로 나온다.

`-` (수식미리보기) ‘Perferences’ \> ‘Packages Setting’ \> ‘LaTeXTools’
\> ‘Settings-User’를 선택한다. ’93번째라인’에 ’preview_math_mode’를
“all”로 바꾼다. 그러면 수식들이 미리 출력된다. 그외에도 자유롭게 셋팅을
조정할 수 있다. 원래셋팅은 ’Perferences’ \> ‘Packages Setting’ \>
‘LaTeXTools’ \> ‘Settings-Defaults’ 에 있다.